In [1]:
import pandas as pd
import numpy as np
import openai
import os
import huggingface_hub
from langchain.llms import OpenAI
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import streamlit as st

In [30]:
open_api_key=""

In [31]:
openai.api_key=open_api_key

In [42]:
from sqlalchemy import create_engine

In [57]:
print(sqlalchemy.__version__)

1.4.39


In [44]:
from langchain_community.vectorstores.faiss import FAISS

In [16]:
from langchain.vectorstores.cassandra import Cassandra
# from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# To improt dataset form huggingface
from datasets import load_dataset

import cassio

In [20]:
from PyPDF2 import PdfReader

In [21]:
astra_db_application_token=""
astra_db_id=""



In [22]:
#reading the pdf
pdfreader=PdfReader("VedantaIR2022-040822.pdf")

In [24]:
#reading the pdf which was imported and taking content/text from it
from typing_extensions import Concatenate
raw_text=""

for i, page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content

In [25]:
#Initializing connection to database
cassio.init(token=astra_db_application_token,database_id=astra_db_id)

In [32]:
#Initalizing open ai api key and embeddings
llm=OpenAI(api_key=open_api_key)
embedding=OpenAIEmbeddings(openai_api_key=open_api_key)

In [33]:
#Creating Langchain Vectorstore backed by Astra DB

astra_vector_store=Cassandra(embedding=embedding,table_name="qa_mini_demo",session=None,keyspace=None)

In [39]:
# Splitting the data into chunks
from langchain.text_splitter import CharacterTextSplitter

# we need to split the text, such that it would not increase the token size
text_splitter=CharacterTextSplitter(separator="\n",chunk_size=800,chunk_overlap=200,length_function=len,)
text=text_splitter.split_text(raw_text)

In [45]:
#Load the dataset into vector store
astra_vector_store.add_texts(text[:50])

astra_vector_index=FAISS.from_texts(texts=text[:50],embedding=embedding)

In [ ]:
first_question=True
while True:
    if first_question:
        query_text=input("\n Enter your question (or type 'quit' to exit): ").strip()
    else:
        query_text=input("\n What's your next question( or type 'quit' to exit): ").strip()
    
    if query_text.lower()=="quit":
        break
    if query_text=="":
        continue
    first_question=False
    
    print("\n Question: \"%s\"" % query_text)
    answer=astra_vector_index.search(query=query_text,search_type="mmr")
    print("Answer: \"%s\"\n"% answer)
    
    print("first document by relevance:")
    for doc,score in astra_vector_store.similarity_search_with_score(query_text,k=4):
        print("     [%0.4f] \"%s....\"" % (score,doc.page_content[:84]))